# Medical Imaging Classification (Colorectal histology) using TensorFlow and the Intel® Transfer Learning Tool API

This notebook facilitates implementation of medical imaging classification using Transfer Learning Toolkit. It performs Multi-class texture analysis in colorectal cancer histology dataset. The workflow uses pretrained SOTA models ( RESNET V1.5) from TF hub and transfers the knowledge from a pretrained domain to a different custom domain achieving required accuracy.

## 1. Import dependencies and setup parameters

In [ ]:
import numpy as np
import os
import pickle
import tensorflow as tf
from sklearn.metrics import classification_report

#tlt imports
from tlt.datasets import dataset_factory
from tlt.models import model_factory
from tlt.utils.types import FrameworkType, UseCaseType

from plot_utils import plot_curves

# Specify a directory for the dataset to be downloaded
dataset_dir = os.environ["DATASET_DIR"] if "DATASET_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "dataset")

# Specify a directory for output
output_dir = os.environ["OUTPUT_DIR"] if "OUTPUT_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "output")

print("Dataset directory:", dataset_dir)
print("Output directory:", output_dir)

## 2. Get the model

In this step, we call the Intel Transfer Learning Tool model factory to list supported TensorFlow image classification models. This is a list of pretrained models from TFHub that we tested with our API. Optionally, the verbose=True argument can be added to the print_supported_models function call to get more information about each model (such as the link to TFHub, image size, the original dataset, etc).

In [ ]:
# See a list of available models
model_factory.print_supported_models(use_case='image_classification', framework='tensorflow')

#### Option A: Load a model

Next, use the model factory to get one of the models listed in the previous cell. The `get_model` function returns a model object that will later be used for training. By default, resnet_v1_50 is used for training.

In [ ]:
# Get the model
model = model_factory.get_model(model_name="resnet_v1_50", framework="tensorflow")

#### Option B: Load a pretrained checkpoint

Optionally, to continue training using a pretrained checkpoint, the user can specify the path to folder containing __saved_model.pb__. The user can specify the path in __model__ parameter.

_Note: The path is same as saved_model_dir_

In [ ]:
#Load a pretrained checkpoint
model = model_factory.load_model(model_name='resnet_v1_50', 
                                 model='/home/intel/output/resnet_v1_50/1', 
                                 framework='tensorflow', use_case='image_classification')

## 3. Get the dataset
Use dataset __colorectal_histology__ from the TensorFlow Datasets catalog

In [ ]:
dataset = dataset_factory.get_dataset(dataset_dir=dataset_dir,
                                      use_case='image_classification', 
                                      framework='tensorflow',
                                      dataset_name='colorectal_histology',
                                      dataset_catalog='tf_datasets')

print(dataset.info)

print("\nClass names:", str(dataset.class_names))

## 4. Prepare the dataset

Once you have your dataset from Option A or Option B above, use the following cells to preprocess the dataset. We resize the images to match the selected models and batch the images, then split them into training and validation subsets.  Data augmentation can be applied by specifying the augmentations to be applied in __add_aug__ parameter. Supported augmentations are 
1. hvflip - RandomHorizontalandVerticalFlip
2. hflip - RandomHorizontalFlip
3. vflip - RandomVerticalFlip
4. rotate - RandomRotate
5. zoom - RandomZoom

In [ ]:
# Preprocess the dataset with an image size that matches the model and a batch size of 32
batch_size = 32
dataset.preprocess(model.image_size, batch_size=batch_size, add_aug=['hvflip', 'rotate'])

In [ ]:
# Split the dataset into training, validation and test subsets
dataset.shuffle_split(train_pct=.80, val_pct=.10, test_pct=0.10)

## 5. Transfer Learning

This step calls the model's train function with the dataset that was just prepared. The training function will get the TFHub feature vector and add on a dense layer based on the number of classes in the dataset. The model is then compiled and trained based on the number of epochs specified in the argument. With the do_eval paramter set to True by default, this step will also show how the model can be evaluated and will return a list of metrics calculated from the dataset's validation subset.
### Arguments
#### Required
-  **dataset** (ImageClassificationDataset, required): Dataset to use when training the model
-  **output_dir** (str): Path to a writeable directory for checkpoint files
-  **epochs** (int): Number of epochs to train the model (default: 1)
#### Optional
-  **initial_checkpoints** (str): Path to checkpoint weights to load. If the path provided is a directory, the latest checkpoint will be used.
-  **early_stopping** (bool): Enable early stopping if convergence is reached while training at the end of each epoch. (default: False)
-  **lr_decay** (bool): If lr_decay is True and do_eval is True, learning rate decay on the validation loss is applied at the end of each epoch.
-  **enable_auto_mixed_precision** (bool or None): Enable auto mixed precision for training. Mixed precision uses both 16-bit and 32-bit floating point types to make training run faster and use less memory. It is recommended to enable auto mixed precision training when running on platforms that support bfloat16 (Intel third or fourth generation Xeon processors). If it is enabled on a platform that does not support bfloat16, it can be detrimental to the training performance. If enable_auto_mixed_precision is set to None, auto mixed precision will be automatically enabled when running with Intel fourth generation Xeon processors, and disabled for other platforms.
-  **extra_layers** (list[int]): Optionally insert additional dense layers between the base model and output layer. This can help increase accuracy when fine-tuning a TFHub model. The input should be a list of integers representing the number and size of the layers, for example [1024, 512] will insert two dense layers, the first with 1024 neurons and the second with 512 neurons.

Note: refer to release documentation for an up-to-date list of train arguments and their current descriptions

In [ ]:
# Mixed precision uses both 16-bit and 32-bit floating point types to make training run faster and use less memory.
# It is recommended to enable auto mixed precision training when running on platforms that support
# bfloat16 (Intel third or fourth generation Xeon processors). If it is enabled on a platform that
# does not support bfloat16, it can be detrimental to the training performance.
# If enable_auto_mixed_precision is set to None, auto mixed precision will be automatically enabled when
# running with Intel fourth generation Xeon processors, and disabled for other platforms.
enable_auto_mixed_precision = None

# Train the model using the dataset
history = model.train(dataset, output_dir=output_dir, epochs=50, 
                      enable_auto_mixed_precision=None, early_stopping=True)

## 6. Evaluate

The next step shows how the model can be evaluated. The model's evaluate function returns a list of metrics calculated from the dataset's validation subset.

In [ ]:
# Evaluate model on validation subset
val_loss, val_acc = model.evaluate(dataset)
print("Validation Accuracy :", val_acc)
print("Validation Loss :", val_loss)

In [ ]:
plot_curves(history, os.path.join(output_dir, "{}_checkpoints".format(model.model_name)))
pickle.dump(history, open(os.path.join(output_dir, "{}_checkpoints".format(model.model_name), 'hist.pkl'), 'wb'))

## 7. Export

Next, we can call the model export function to generate a saved_model.pb. The model is saved in a format that is ready to use with TensorFlow Serving. Each time the model is exported, a new numbered directory is created, which allows serving to pick up the latest model.

In [ ]:
saved_model_dir = model.export(output_dir)

## 8. Inference
To perform only Inference using a saved model, follow the steps below
1. Execute Step 2(b) to load a pretrained checkpoint with the appropriate model name.
2. Execute Steps 3 and 4 to load and prepare the dataset.
3. Continue with the steps below

In [ ]:
history = pickle.load(open(os.path.join(output_dir, "{}_checkpoints".format(model.model_name), 'hist.pkl'), 'rb'))
plot_curves(history, os.path.join(output_dir, "{}_checkpoints".format(model.model_name)))

In [ ]:
loss, accuracy = model.evaluate(dataset, use_test_set=True)
print('Test accuracy :', accuracy)

We get the test subset from our dataset, and use that to call predict on our model.

In [ ]:
actual_labels = np.concatenate([y for x, y in dataset._test_subset], axis=0)
predicted_labels = model.predict(dataset._test_subset)
report = classification_report(actual_labels, predicted_labels)
print("Classification report")
print(report)

## Dataset Citations

@article{kather2016multi,<br>
      title={Multi-class texture analysis in colorectal cancer histology},<br>
      author={Kather, Jakob Nikolas and Weis, Cleo-Aron and Bianconi, Francesco and Melchers, Susanne M and Schad, Lothar R and Gaiser, Timo and Marx, Alexander and Z{"o}llner, Frank Gerrit},<br>
      journal={Scientific reports},<br>
      volume={6},<br>
      pages={27988},<br>
      year={2016},<br>
      publisher={Nature Publishing Group}<br>
    }
    
Kather, J. N., Zöllner, F. G., Bianconi, F., Melchers, S. M., Schad, L. R., Gaiser, T., Marx, A., & Weis, C.-A. (2016). Collection of textures in colorectal cancer histology [Data set]. Zenodo. https://doi.org/10.5281/zenodo.53169